In [32]:
from archappl.client import ArchiverDataClient
from archappl.data import epoch_to_iso, iso_to_epoch
from phantasy import MachinePortal
import numpy as np

In [20]:
mp = MachinePortal("FRIB", "LEBT")
lat = mp.work_lattice_conf

valid_dtypes = ('SOL', 'VCOR', 'HCOR', 'EQUAD',)

INFO: 2019-06-18 15:34:14,617: phantasy.library.parser.config: Searching configuration in relative or absolute path: 'FRIB'
INFO: 2019-06-18 15:34:14,618: phantasy.library.parser.config: Searching configuration under path: '/home/tong/Dropbox/FRIB/work/phantasy-project/phantasy-machines' 'FRIB'
INFO: 2019-06-18 15:34:14,620: phantasy.library.parser.config: Importing 'FRIB' from '/home/tong/Dropbox/FRIB/work/phantasy-project/phantasy-machines/FRIB'
INFO: 2019-06-18 15:34:14,622: phantasy.library.parser.config: Using config file: phantasy.ini
INFO: 2019-06-18 15:34:14,700: phantasy.library.operation.lattice: Using CSV/SQLite instead of CFS '/home/tong/Dropbox/FRIB/work/phantasy-project/phantasy-machines/FRIB/./linac/all_channels.csv'
INFO: 2019-06-18 15:34:14,701: phantasy.library.pv.datasource: Set source to be CSV table: all_channels.csv.
INFO: 2019-06-18 15:34:14,898: phantasy.library.operation.lattice: Model data directory: /home/tong/phantasy_data/20190618/model_data/data_153414
INF

In [21]:
s = {}
for elem in lat:
    if elem.family in valid_dtypes:
        f = elem.get_eng_fields()[0]
        sp_pvs = elem.pv(handle='setpoint', field=f)
        s.update({k:100 * np.random.random() + 1 for k in sp_pvs})

In [22]:
quad = lat.get_elements(type='EQUAD')[0]
sol = lat.get_elements(type='SOL')[0]

In [23]:
print(quad.get_settings('V', s))
print(sol.get_settings('I', s))

3.5057602449582177
61.00791797825785


### Read settings from snp file

In [24]:
snpfile = '/home/tong/Dropbox/FRIB/work/phantasy-project/phantasy-apps/phantasy_apps/settings_manager/testdata/Ar_LEBT_to_MEBT_20180321.snp'

In [25]:
from phantasy import generate_settings
from phantasy import snp2dict

In [8]:
settings = generate_settings(snpfile, lat, only_physics=False)

In [9]:
settings.write('/tmp/settings2.json')

In [10]:
from phantasy import get_element_settings

In [16]:
get_element_settings(s, sol, only_physics=False)

{'B': -0.027918183210821942, 'I': 9.660271007204823}

In [17]:
settings_dict = snp2dict(snpfile=snpfile)

In [13]:
quad.get_settings('V', settings_dict)

3072.0

In [14]:
quad.get_settings('V', settings_dict)

3072.0

In [18]:
get_element_settings(s, quad, only_physics=False)

{'V': 2.4933180597827516}

### Read settings from AA

In [27]:
pv_set = set()
for elem in lat:
    if elem.family in valid_dtypes:
        pv_set.update(elem.pv(handle='setpoint'))

In [31]:
client = ArchiverDataClient()
client.url = "http://epicsarchiver0.ftc:17668"

In [ ]:
ts_iso = "2019-03-05T13:04:08.120000-05:00"
client.get_data_at_time(pv_set, ts=ts_iso)